In [280]:
import numpy as np
import h5py

In [281]:
a = np.random.randint(0, 100, (5, 11))

In [282]:
d = a[np.newaxis, np.newaxis, :, :]
c = np.linspace(5.5, -5.5, 12)

In [283]:
f = h5py.File('flood.h5','w')
f.create_dataset('entry1/data/hmm', data=d)
f.create_dataset('entry1/data/x_bin', data=c)
f.close()

In [284]:
def create_detector_norm(h5norm, x_min, x_max):
    """
    Produces a detector normalisation array for Platypus.
    Here we average over N, T and X to provide  a relative efficiency for each
    y wire.

    Parameters
    ----------
    h5norm : hdf5 file
        Containing a flood field run (water)
    x_min : float, int
        Minimum x location to use
    x_max : float, int
        Maximum x location to use

    Returns
    -------
    norm, norm_sd : array_like
        1D array containing the normalisation data for each y pixel
    """
    # sum over N and T
    detector = h5norm['entry1/data/hmm']
    x_bin = h5norm['entry1/data/x_bin']

    # find out what pixels to use
    x_low = np.searchsorted(x_bin, x_min, sorter=np.argsort(x_bin))
    x_low = np.argsort(x_bin)[x_low]
    x_high = np.searchsorted(x_bin, x_max, sorter=np.argsort(x_bin))
    x_high = np.argsort(x_bin)[x_high]

    if(x_low > x_high):
        x_low, x_high = x_high, x_low

    norm = np.sum(detector, axis=(0, 1),
                  dtype='float64')

    # By this point you have norm[y][x]
    norm = norm[:, x_low: x_high]
    print(x_low, x_high, norm.shape)
    norm = np.sum(norm, axis=1)
    print(norm.shape)
    mean = np.mean(norm)

    return norm / mean, np.sqrt(norm) / mean

In [293]:
h5norm = h5py.File('flood.h5', 'r')
print(create_detector_norm(h5norm, 3.5, -3.5))
h5norm.close()

2 9 (5, 7)
(5,)
(array([ 1.12290503,  1.23743017,  0.8603352 ,  0.70111732,  1.07821229]), array([ 0.05600541,  0.05879208,  0.04902215,  0.04425413,  0.05487956]))


In [286]:
d

array([[[[31, 46, 19, 89, 87, 33, 21, 81, 72, 90, 20],
         [72, 58,  6, 67, 82, 26, 99, 86, 77, 14, 28],
         [16, 12, 40, 81, 97, 22,  8,  8, 52, 23, 11],
         [63, 48, 68, 66, 38,  3, 11, 34, 31, 98, 34],
         [98, 19, 74,  0, 73, 32, 88, 99, 20, 70, 64]]]])

In [294]:
np.sum(np.squeeze(d[..., 2:-2]), axis=1) / np.mean(np.sum(np.squeeze(d[..., 2:-2]), axis=1))

array([ 1.12290503,  1.23743017,  0.8603352 ,  0.70111732,  1.07821229])

In [295]:
pwd

'/Users/anz/Documents/Andy/programming/refnx'